In [1]:
import json
from pathlib import Path
from .. import paths

In [2]:
#ground truth file artpedia2wiki_combined.json
path_gt_annotations=paths.COMBINED_ANNOTATIONS_PATH
path_falcon_annotations=Path("falcon_res.json")

In [3]:
gt_annotations=json.load(open(path_gt_annotations))
falcon_annotations=json.load(open(path_falcon_annotations))

In [4]:
#limit the gt to the test set
gt_annotations={k:v for k,v in gt_annotations.items() if v["split"]=="test"}
len(gt_annotations)

100

In [5]:
def find_key(query, keys):
    if query in keys:
        return query
    #find a key that starts with or ends with query
    for key in keys:
        if key.startswith(query) or key.endswith(query):
            return key
    #find the other way around
    for key in keys:
        if query.startswith(key) or query.endswith(key):
            return key
    #find a key that contains query
    for key in keys:
        if query in key:
            return key
    return None


In [6]:
def group_falcon_mentions(falcon_mentions):
    mentions_groups={}
    current_group=[]
    current_key=None
    for mention in falcon_mentions:
        if current_key==mention["surface form"]:
            current_group.append(mention)
        if current_key!=mention["surface form"]:
            if current_key!=None:
                mentions_groups[current_key]=current_group
            current_group=[]
            current_group.append(mention)
            current_key=mention["surface form"]
        if len(current_group)==50:
            mentions_groups[current_key]=current_group
            current_group=[]
    if len(current_group)>0:
        mentions_groups[current_key]=current_group
    return mentions_groups

ranks=[]
for qid,sentences in falcon_annotations.items():
    for falcon_sentence in sentences:
        falcon_text=falcon_sentence["text"]
        falcon_mentions=falcon_sentence["falcon"]["entities_wikidata"] # list of dictionaries, each dictionary has this structure { "URI": "http://www.wikidata.org/entity/Q18627384","surface form": "Crucifixion"}
        #split the mentions in groups of 50, take the surface form as key
        mentions_groups=group_falcon_mentions(falcon_mentions)
        #find text in gt_annotations
        gt_object=gt_annotations[qid]
        #check if the text is in the gt_annotations visual_sentences
        gt_mentions=None
        visual_sentences=gt_object["visual_sentences"]
        try:
            falcon_sentence_index=visual_sentences.index(falcon_text)
            gt_mentions=gt_object["visual_el_matches"][falcon_sentence_index]
        except ValueError:
            #try with the contextual sentences
            contextual_sentences=gt_object["contextual_sentences"]
            try:
                falcon_sentence_index=contextual_sentences.index(falcon_text)
                gt_mentions=gt_object["contextual_el_matches"][falcon_sentence_index]
            except ValueError:
                print("Sentence not found in visual or contextual sentences")
                break
        # print("Falcon text: ",falcon_text)
        # print("Falcon mentions: ",mentions_groups)
        # print("GT mentions: ",gt_mentions)
        for gt_mention in gt_mentions:
            qid_gt=gt_mention["qid"] # http....
            surface_form_gt=gt_mention["text"]
            #find surface form in falcon_mentions
            key=find_key(surface_form_gt,mentions_groups.keys())
            if key:
                falcon_mentions=mentions_groups[key]
                for falcon_mention in falcon_mentions:
                    qid_falcon=falcon_mention["URI"]
                    if qid_falcon==qid_gt:
                        try:
                            rank=falcon_mentions.index(falcon_mention)+1
                            ranks.append(rank)
                        except ValueError:
                            ranks.append(None)
            else:
                ranks.append(None)
ranks[:10]    

In [8]:
ranks=[]
el_matches_length=0
for qid,gt_annotation in gt_annotations.items():
    visual_sentences=gt_annotation["visual_sentences"]
    visual_el_matches=gt_annotation["visual_el_matches"]
    contextual_sentences=gt_annotation["contextual_sentences"]
    contextual_el_matches=gt_annotation["contextual_el_matches"]
    #concate the visual and contextual sentences
    all_sentences=visual_sentences+contextual_sentences
    all_el_matches=visual_el_matches+contextual_el_matches
    el_matches_length+=sum(len(el_matches) for el_matches in all_el_matches)
    for sentence,el_matches in zip(all_sentences,all_el_matches):
        sentence_ranks=[]
        #find the sentence in falcon_annotations
        falcon_sentence=None
        for falcon_sentences in falcon_annotations[qid]:
            if falcon_sentences["text"]==sentence:
                falcon_sentence=falcon_sentences
                break
        if falcon_sentence:
            falcon_mentions=falcon_sentence["falcon"]["entities_wikidata"]
            mentions_groups=group_falcon_mentions(falcon_mentions)
            for el_match in el_matches:
                qid_gt=el_match["qid"]
                surface_form_gt=el_match["text"]
                key=find_key(surface_form_gt,mentions_groups.keys())
                if key:
                    falcon_mentions=mentions_groups[key]
                    uri_falcon=[falcon_mention["URI"] for falcon_mention in falcon_mentions]
                    try:
                        rank=uri_falcon.index(qid_gt)+1
                        sentence_ranks.append(rank)
                    except ValueError:
                        sentence_ranks.append(None)
                    # for falcon_mention in falcon_mentions:
                    #     qid_falcon=falcon_mention["URI"]
                    #     if qid_falcon==qid_gt:
                    #         try:
                    #             rank=falcon_mentions.index(falcon_mention)+1
                    #             sentence_ranks.append(rank)
                    #         except ValueError:
                    #             sentence_ranks.append(None)
                else:
                    sentence_ranks.append(None)
        else:
            if el_matches:
                for el_match in el_matches:
                    sentence_ranks.append(None)
                print("Sentence not found in falcon_annotations")
        assert len(sentence_ranks)==len(el_matches)
        ranks.extend(sentence_ranks)
ranks[:10]    

[None, None, None, None, 1, None, None, None, None, None]

In [9]:
len(ranks)

634

In [10]:
el_matches_length

634

In [12]:
k_values=[1,3,5,10,20]
for k in k_values:
    #compute hits@k
    hits_k=sum([1 for rank in ranks if rank and rank<=k])
    print(f"hits@{k}={hits_k/el_matches_length}")

hits@1=0.09305993690851735
hits@3=0.13564668769716087
hits@5=0.15772870662460567
hits@10=0.16561514195583596
hits@20=0.19558359621451105


In [13]:
#number of mentions that are not found in falcon_annotations
sum([1 for rank in ranks if rank==None])

510

In [14]:
sum([1 for rank in ranks if rank==None])/el_matches_length

0.804416403785489